### Compare Earliest Dates
From EDA it is apparent that frequently fundamental data is available before the stock was traded on the exchange. This is likely because initially most companies are privately held before going public. Code compares earliest dates.

In [2]:
import operator
import math
import random
import pandas as pd
import datetime as dt
import numpy as np
import os
import urllib
import json
import requests
from io import StringIO
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
from eod import EodHistoricalData
#import pandas_datareader as pdr
#import yfinance as yf
#import talib

from functools import partial


from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

pd.set_option('display.max_rows', None)

In [3]:
#paths for data - set prefix to location of Data folder
path_prefix = r'C:\Users\OEM\GDrive\WQU'
path_att = r'\Data\fundamentals_by_attribute'
path_fun = r'\Data\fundamentals_by_ticker'
path_std = r'\Data\standardised_fundamentals'
path_eda = r'\Data\exploratory_data_analysis'
path_rob = r'\Data\robust_scaling'

In [4]:
api_key = "618f834b7bfe27.18132752"
client = EodHistoricalData(api_key)
resp = client.get_exchange_symbols(exchange='NYSE')

In [6]:
tickers = []
for i in range(len(resp)):
    if resp[i]['Type'] == 'Common Stock' or resp[i]['Type'] == 'Preferred Stock' :
        ticker = resp[i]['Code']
        tickers.append(ticker)

In [ ]:
df_dat = pd.DataFrame(columns=['first_filing', 'first_traded', 'timedelta'])
count = 0
for ticker in tickers:
    filepath = Path(path_prefix + path_fun + '/df_fun_{}.csv'.format(ticker))
    if os.path.isfile(filepath) == True:
        df_fun = pd.read_csv(filepath, index_col=0).transpose()
        idx = pd.to_datetime(df_fun.index)
        df_fun.set_index(idx, inplace=True)
        fun_earliest = df_fun.last_valid_index()
        df_dat.loc[ticker, 'first_filing'] = fun_earliest
        
        resp = client.get_prices_eod(ticker + ".US")
        df_prices = pd.DataFrame(resp)
        idx = pd.to_datetime(df_prices['date'])
        df_prices.set_index(idx, inplace=True)
        df_prices.sort_index(ascending=False, inplace=True)
        price_earliest = df_prices.last_valid_index()
        df_dat.loc[ticker, 'first_traded'] = price_earliest
        delta = fun_earliest - price_earliest
        df_dat.loc[ticker, 'timedelta'] = delta

        print(ticker)
        
    count = count + 1
    if count % 100 == 0:
        df_dat.to_csv(path_prefix + "/" + "earliest_dates.csv")


In [7]:
df_dat = pd.read_csv(path_prefix + "/" + "earliest_dates.csv", index_col=0)

In [11]:
df_dat

RSG            -1 days +00:00:00
UPH            -1 days +00:00:00
LCTX         -100 days +00:00:00
VGR          -100 days +00:00:00
SNX         -1000 days +00:00:00
BBW         -1001 days +00:00:00
BAC-PN     -10025 days +00:00:00
PBI-PB     -10028 days +00:00:00
C-PJ       -10031 days +00:00:00
YALA        -1004 days +00:00:00
ALTG-PA     -1006 days +00:00:00
C-PK       -10080 days +00:00:00
ALL-PI     -10080 days +00:00:00
WFC-PR     -10119 days +00:00:00
EFC         -1012 days +00:00:00
IH          -1013 days +00:00:00
PSFE        -1013 days +00:00:00
TMHC        -1015 days +00:00:00
DLNG-PA     -1017 days +00:00:00
ZGN         -1017 days +00:00:00
STT-PD     -10195 days +00:00:00
TKC          -102 days +00:00:00
USAC        -1021 days +00:00:00
KODK       -10220 days +00:00:00
MS-PE      -10223 days +00:00:00
SWI         -1023 days +00:00:00
EE         -10240 days +00:00:00
COLD        -1025 days +00:00:00
RF-PB      -10255 days +00:00:00
GHLD        -1026 days +00:00:00
MNR       